<a href="https://colab.research.google.com/github/zhannatoleubek-png/special-okx-chainsaw/blob/main/%D0%BB%D1%83%D1%87%D1%88%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================================================
# 🚀 OKX Impulse Scanner + Vumanchu Cipher B Logic (Colab Ready)
# Добавлено: BTC multi-TF sentiment + SHORT signals (inverted logic) + BTC Dominance Display
# =====================================================
!pip install --quiet ccxt pandas numpy requests ta

import time
import datetime as dt
import requests
import pandas as pd
import numpy as np
import ccxt
from ta.momentum import RSIIndicator
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import EMAIndicator

# ---------------- CONFIG ----------------
CONFIG = {
    "telegram_bot_token": "8288179092:AAGczGGxYbHc0SqDrKO_BSayAZv1PL7ufJU",
    "telegram_chat_id": "381202205",
    "symbols": [],
    "scan_interval": 10 * 60,
    "atr_period": 14,
    "min_confidence_score": 6.0,
    "min_base_volume_24h": 100000
}

# =====================================================
# ⚙️ HELPERS
# =====================================================
def send_telegram_message(bot_token, chat_id, text, parse_mode="HTML"):
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {"chat_id": chat_id, "text": text, "parse_mode": parse_mode, "disable_web_page_preview": True}
    try:
        r = requests.post(url, json=payload, timeout=10)
        print("📨 Telegram response:", r.status_code)
        return r.json()
    except Exception as e:
        print("❌ Telegram send error:", e)
        return None


def now_ts():
    return dt.datetime.now(dt.timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")


exchange = ccxt.okx({"enableRateLimit": True})


def safe_fetch_ohlcv(symbol, timeframe='1h', limit=200):
    try:
        return exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    except Exception as e:
        print(f"⚠ safe_fetch_ohlcv error {symbol} {timeframe}: {e}")
        return None


# =====================================================
# ⚙️ Custom Indicators
# =====================================================
def compute_vwap(df):
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    pv = typical_price * df['vol']
    vol_cum = df['vol'].cumsum().replace(0, np.nan)
    vwap = pv.cumsum() / vol_cum
    vwap = vwap.fillna((pv.cumsum() / (df['vol'].cumsum().replace(0, np.nan))).fillna(method='bfill').fillna(method='ffill'))
    return vwap


def money_flow_index(high, low, close, volume, window=14):
    typical_price = (high + low + close) / 3
    money_flow = typical_price * volume
    pos, neg = [0], [0]
    for i in range(1, len(typical_price)):
        if typical_price[i] > typical_price[i - 1]:
            pos.append(money_flow[i]); neg.append(0)
        elif typical_price[i] < typical_price[i - 1]:
            pos.append(0); neg.append(money_flow[i])
        else:
            pos.append(0); neg.append(0)
    pos_mf = pd.Series(pos).rolling(window).sum()
    neg_mf = pd.Series(neg).rolling(window).sum()
    mfi = 100 * (pos_mf / (pos_mf + neg_mf).replace(0, np.nan))
    return mfi.fillna(50.0)


def wavetrend(df, chlen=10, avg=21):
    df = df.copy()
    df['hlc3'] = (df['high'] + df['low'] + df['close']) / 3
    df['esa'] = df['hlc3'].ewm(span=chlen, adjust=False).mean()
    df['d'] = abs(df['hlc3'] - df['esa']).ewm(span=chlen, adjust=False).mean()
    df['ci'] = (df['hlc3'] - df['esa']) / (0.015 * df['d'].replace(0, np.nan))
    df['tci'] = df['ci'].ewm(span=avg, adjust=False).mean()
    df['wt1'] = df['tci'].fillna(0)
    df['wt2'] = df['wt1'].ewm(span=4, adjust=False).mean().fillna(0)
    return df


# =====================================================
# ₿ BTC Sentiment + Dominance
# =====================================================
def get_btc_dominance():
    """Возвращает текущую доминацию BTC (%)"""
    try:
        url = "https://api.coinmarketcap.com/data-api/v3/global-metrics/quotes/latest"
        data = requests.get(url, timeout=10).json()
        btc_dom = float(data["data"]["btcDominance"])
        return btc_dom
    except Exception as e:
        print("⚠ BTC Dominance fetch error:", e)
        return None


def get_btc_sentiment():
    symbol = 'BTC/USDT'
    tf_primary = ['1d', '12h', '4h', '1h', '30m']
    tf_short = ['15m', '5m', '3m', '1m']

    def tf_dir(tf):
        o = safe_fetch_ohlcv(symbol, tf, limit=220)
        if not o:
            return None
        df = pd.DataFrame(o, columns=['ts', 'open', 'high', 'low', 'close', 'vol'])
        df['close'] = df['close'].astype(float)
        try:
            ema50 = EMAIndicator(df['close'], 50).ema_indicator().iloc[-1]
            ema200 = EMAIndicator(df['close'], 200).ema_indicator().iloc[-1]
            price = df['close'].iloc[-1]
            if price > ema50 and price > ema200:
                return 'bull'
            elif price < ema50 and price < ema200:
                return 'bear'
            else:
                return 'neutral'
        except Exception:
            return None

    def majority(dirs):
        cnt = {'bull': 0, 'bear': 0, 'neutral': 0}
        for d in dirs:
            if d in cnt:
                cnt[d] += 1
        best = max(cnt.items(), key=lambda x: x[1])
        if best[1] == 0:
            return 'neutral'
        vals = list(cnt.values())
        if vals.count(best[1]) > 1:
            return 'neutral'
        return best[0]

    prim_dirs = [tf_dir(tf) for tf in tf_primary]
    short_dirs = [tf_dir(tf) for tf in tf_short]
    primary_sent = majority(prim_dirs)
    short_sent = majority(short_dirs)

    btc_dom = get_btc_dominance()
    dom_text = f"BTC Dominance: {btc_dom:.2f}%" if btc_dom is not None else "BTC Dominance: N/A"

    prim_summary = ", ".join([f"{tf}:{(d or '?')}" for tf, d in zip(tf_primary, prim_dirs)])
    short_summary = ", ".join([f"{tf}:{(d or '?')}" for tf, d in zip(tf_short, short_dirs)])
    summary = f"₿ BTC Trend — Primary({prim_summary}) | Short({short_summary})\nPrimary sentiment: {primary_sent.upper()}, Short sentiment: {short_sent.upper()}\n{dom_text}"

    return summary, primary_sent, short_sent


# =====================================================
# ⚙️ Vumanchu Cipher B + EMA + VWAP + RSI + MFI + Volume Logic
# =====================================================
def analyze_vumanchu_cipher(symbol):
    btc_summary, btc_primary_sent, btc_short_sent = get_btc_sentiment()

    ohlcv_1h = safe_fetch_ohlcv(symbol, '1h', 200)
    ohlcv_15m = safe_fetch_ohlcv(symbol, '15m', 200)
    ohlcv_3m = safe_fetch_ohlcv(symbol, '3m', 300)
    if not ohlcv_1h or not ohlcv_15m or not ohlcv_3m:
        return None

    df1h = pd.DataFrame(ohlcv_1h, columns=['ts','open','high','low','close','vol'])
    df3 = pd.DataFrame(ohlcv_3m, columns=['ts','open','high','low','close','vol'])
    for df in [df1h, df3]:
        df[['close','high','low','vol']] = df[['close','high','low','vol']].astype(float)

    ema50_1h = EMAIndicator(df1h['close'], 50).ema_indicator()
    ema200_1h = EMAIndicator(df1h['close'], 200).ema_indicator()
    trend_1h_bull = df1h['close'].iloc[-1] > ema50_1h.iloc[-1] and df1h['close'].iloc[-1] > ema200_1h.iloc[-1]
    trend_1h_bear = df1h['close'].iloc[-1] < ema50_1h.iloc[-1] and df1h['close'].iloc[-1] < ema200_1h.iloc[-1]

    df3 = wavetrend(df3)
    df3['ema8'] = EMAIndicator(df3['close'], 8).ema_indicator()
    df3['ema21'] = EMAIndicator(df3['close'], 21).ema_indicator()
    df3['ema50'] = EMAIndicator(df3['close'], 50).ema_indicator()
    df3['rsi'] = RSIIndicator(df3['close'], 14).rsi()
    df3['mfi'] = money_flow_index(df3['high'], df3['low'], df3['close'], df3['vol'])
    df3['obv'] = OnBalanceVolumeIndicator(df3['close'], df3['vol']).on_balance_volume()
    df3['vwap'] = compute_vwap(df3)

    last, prev = df3.iloc[-1], df3.iloc[-2]

    wt_cross_up = (prev['wt1'] < prev['wt2']) and (last['wt1'] > last['wt2'])
    ema_bull = (last['ema8'] > last['ema21']) and (last['close'] > last['ema8'])
    rsi_ok = last['rsi'] > 50 and last['rsi'] > prev['rsi']
    mfi_ok = last['mfi'] > 50
    vol_ok = last['vol'] > df3['vol'].rolling(20).mean().iloc[-1]*1.1

    wt_cross_down = (prev['wt1'] > prev['wt2']) and (last['wt1'] < last['wt2'])
    ema_bear = (last['ema8'] < last['ema21']) and (last['close'] < last['ema8'])
    rsi_down = last['rsi'] < 50 and last['rsi'] < prev['rsi']
    mfi_down = last['mfi'] < 50
    vol_down = last['vol'] > df3['vol'].rolling(20).mean().iloc[-1]*1.1

    long_score = sum([wt_cross_up, ema_bull, rsi_ok, mfi_ok, vol_ok])
    short_score = sum([wt_cross_down, ema_bear, rsi_down, mfi_down, vol_down])

    long_allowed = btc_short_sent != 'bear' and not trend_1h_bear
    short_allowed = btc_short_sent != 'bull' and not trend_1h_bull

    if long_allowed and long_score >= 4:
        sl = float(df3['low'].rolling(5).min().iloc[-1])*0.997
        risk = last['close'] - sl
        tp = last['close'] + 2*risk
        signal_type = "STRONG LONG" if long_score == 6 else "LONG"
        msg = (
            f"{btc_summary}\n\n"
            f"🔔 <b>{symbol}</b> — {signal_type}\n"
            f"Entry: {last['close']:.6f}\nSL: {sl:.6f}\nTP: {tp:.6f}\n"
            f"{now_ts()}"
        )
        send_telegram_message(CONFIG['telegram_bot_token'], CONFIG['telegram_chat_id'], msg)

    if short_allowed and short_score >= 4:
        sl = float(df3['high'].rolling(5).max().iloc[-1])*1.003
        risk = sl - last['close']
        tp = last['close'] - 2*risk
        signal_type = "STRONG SHORT" if short_score == 6 else "SHORT"
        msg = (
            f"{btc_summary}\n\n"
            f"🔔 <b>{symbol}</b> — {signal_type}\n"
            f"Entry: {last['close']:.6f}\nSL: {sl:.6f}\nTP: {tp:.6f}\n"
            f"{now_ts()}"
        )
        send_telegram_message(CONFIG['telegram_bot_token'], CONFIG['telegram_chat_id'], msg)


# =====================================================
# 🔁 MAIN LOOP
# =====================================================
def main_loop(cfg):
    symbols = cfg['symbols'] or [s for s in exchange.load_markets() if 'USDT' in s]
    print(f"✅ Found {len(symbols)} filtered symbols")
    while True:
        print(f"{now_ts()} — scanning {len(symbols)} symbols...")
        for i, sym in enumerate(symbols, 1):
            try:
                print(f"[{i}/{len(symbols)}] Scanning {sym}...")
                analyze_vumanchu_cipher(sym)
            except Exception as e:
                print(f"❌ {sym} analysis error:", e)
        print(f"{now_ts()} — waiting {cfg['scan_interval']} sec...\n")
        time.sleep(cfg['scan_interval'])


# =====================================================
# ▶️ RUN
# =====================================================
main_loop(CONFIG)


✅ Found 559 filtered symbols
2025-10-21 19:52:20 UTC — scanning 559 symbols...
[1/559] Scanning USDT/SGD...


/tmp/ipython-input-3233841285.py:66: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vwap = vwap.fillna((pv.cumsum() / (df['vol'].cumsum().replace(0, np.nan))).fillna(method='bfill').fillna(method='ffill'))


[2/559] Scanning USDT/AUD...


/tmp/ipython-input-3233841285.py:66: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vwap = vwap.fillna((pv.cumsum() / (df['vol'].cumsum().replace(0, np.nan))).fillna(method='bfill').fillna(method='ffill'))


[3/559] Scanning USDT/AED...


/tmp/ipython-input-3233841285.py:66: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vwap = vwap.fillna((pv.cumsum() / (df['vol'].cumsum().replace(0, np.nan))).fillna(method='bfill').fillna(method='ffill'))


[4/559] Scanning USDT/BRL...


/tmp/ipython-input-3233841285.py:66: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vwap = vwap.fillna((pv.cumsum() / (df['vol'].cumsum().replace(0, np.nan))).fillna(method='bfill').fillna(method='ffill'))


[5/559] Scanning USDT/EUR...
